In [151]:
import numpy as np 
import matplotlib.pyplot as plt
import math
import torch
import torch.nn.functional as F

def Y_L(x):
#     if (x<0.5):
#         return (0.5*(6*x-2)*(6*x-2)*math.sin(12*x-4))+(10*(x-0.5))-5
#     else:
#         return 3+(0.5*(6*x-2)*(6*x-2)*math.sin(12*x-4))+(10*(x-0.5))-5
    
    return 0.5*(6*x-2)*(6*x-2)*math.sin(12*x-4)+10*(x-0.5)-5
def Y_H(x):
#     if (x<0.5):
#         return 2*Y_L(x)-20*x+20
#     else:
#         return 4+2*Y_L(x)-20*x+20
    
    return (6*x-2)*(6*x-2)*math.sin(12*x-4)
H_x = []
H_y = []
H_yl = []
for i in range(0,100,1):
    H_x.append(i/100)
    H_y.append(Y_H(i/100))
    H_yl.append(Y_L(i/100))

H_training_input = np.array([0.2,0.4,0.6,0.8,0.9])
output_list = []     ## output list of high fidelity 
output_list_l = []   ## output list of low fidelity 
for i in range(0,5,1):
    output_list.append(Y_H(H_training_input[i]))
    output_list_l.append(Y_L(H_training_input[i]))
H_training_output = np.asarray(output_list)
H_training_output_l = np.asarray(output_list_l)

In [152]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 5, 1, 20, 1

H_training_input1 = H_training_input.reshape((5,1))
H_training_input2 = H_training_input1.transpose()

H_training_output1 = H_training_output.reshape((5,1))
H_training_output2 = H_training_output1.transpose()

H_training_output1_l = H_training_output_l.reshape((5,1))
H_training_output2_l = H_training_output1_l.transpose()

print(type(H_training_input),'###',H_training_input.shape,H_training_input2.shape)

x1 = torch.from_numpy(H_training_input2).float()
y1 = torch.from_numpy(H_training_output2).float()
y1_l = torch.from_numpy(H_training_output2_l).float()

x = x1.t()
y = y1.t()
y_l = y1_l.t()


<class 'numpy.ndarray'> ### (5,) (1, 5)


In [153]:
print("input values",x)
print("input values",y)
print("input values",y_l)

input values tensor([[0.2000],
        [0.4000],
        [0.6000],
        [0.8000],
        [0.9000]])
input values tensor([[-0.6397],
        [ 0.1148],
        [-0.1494],
        [-4.9491],
        [ 5.7120]])
input values tensor([[-8.3199],
        [-5.9426],
        [-4.0747],
        [-4.4746],
        [ 1.8560]])


In [154]:
# pytorch model

%run models_for_multi.ipynb

In [159]:
model1 = DNN_LF(D_in, H, D_out)
model2 = multi_fnn(D_in, H, D_out)

# vanilla stochastic gradient descent is tough, so we use momentum
error = torch.nn.MSELoss()
optimizer = torch.optim.LBFGS(model1.parameters(), lr=0.001)

for i in range(100):
    def closure():
        optimizer.zero_grad()
        output = model1(x)
        loss = error(output, y_l)
        loss.backward()
        return loss
    #print(optimizer.step(closure))
    optimizer.step(closure)

In [161]:
optimizer2 = torch.optim.LBFGS(model2.parameters(), lr=1e-3)

for t in range(100):
    def closure():
        optimizer.zero_grad()
        y_l_p   = model1(x)
        y_h     = model2(y_l_p,x)
        print(y_l_p)
        print(y_h)
        err 
        loss_yh = error(y_h, y)
        #loss = loss_yl + loss_yh
        loss_yh.backward()
        return loss
    optimizer2.step(closure)

tensor([[-0.3137],
        [-0.3286],
        [-0.3403],
        [-0.3489],
        [-0.3521]], grad_fn=<AddBackward0>)


NameError: name 'err' is not defined

In [ ]:
temp = np.asarray(H_x)
temp1 = temp.reshape((100,1))
temp2 = temp1.transpose()
t1 = torch.FloatTensor(temp2)
t = t1.t()

Ttemp  = np.asarray(H_x)
Ttemp1 = Ttemp.reshape((100,1))
Ttemp2 = Ttemp1.transpose()
Tt1    = torch.FloatTensor(Ttemp2)
Tt     = Tt1.t()


p_l = model1(t)
p_h_1 = model2(p_l,t)
p_h_2 = model3(p_l,t)
# p_h_1 = model2(Tt,t)
# p_h_2 = model3(Tt,t)
prediction = alpha * p_h_1 + (1-alpha)*p_h_2

plt.plot(H_x,H_y,H_training_input,H_training_output,'ro',H_x,prediction.tolist())
plt.show()
